# <!-- TITLE --> [IMDB4] - Reload embedded vectors
<!-- DESC --> Retrieving embedded vectors from our trained model
<!-- AUTHOR : Jean-Luc Parouty (CNRS/SIMaP) -->

## Objectives :
 - The objective is to retrieve and visualize our embedded vectors
 - For this, we will use our **previously saved model**.

## What we're going to do :

 - Retrieve our saved model
 - Extract vectors and play with

## Step 1 - Init python stuff

In [2]:
import numpy as np

import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.datasets.imdb as imdb

import matplotlib.pyplot as plt
import matplotlib
import pandas as pd

import os,sys,h5py,json,re

from importlib import reload

run_dir = './run/IMDB2'
datasets_dir = 'fidle-datasets'

### 1.2 - Parameters
The words in the vocabulary are classified from the most frequent to the rarest.  
`vocab_size` is the number of words we will remember in our vocabulary (the other words will be considered as unknown).  
`review_len` is the review length  
`dictionaries_dir` is where we will go to save our dictionaries. (./data is a good choice)

In [3]:
vocab_size           = 10000
review_len           = 256

dictionaries_dir     = './data'

## Step 2 - Get the embedding vectors !

### 2.1 - Load model and dictionaries

In [4]:
model = keras.models.load_model(f'{run_dir}/models/best_model.h5')
print('Model loaded.')

with open(f'{dictionaries_dir}/index_word.json', 'r') as fp:
    index_word = json.load(fp)
    index_word = { int(i):w for i,w in index_word.items() }
    word_index = { w:int(i) for i,w in index_word.items() }
    print('Dictionary loaded.')

Model loaded.
Dictionary loaded.


### 2.2 - Retrieve embeddings

In [5]:
embeddings = model.layers[0].get_weights()[0]
print('Shape of embeddings : ',embeddings.shape)

Shape of embeddings :  (10000, 16)


### 2.3 - Build a nice dictionary

In [6]:
word_embedding = { index_word[i]:embeddings[i] for i in range(vocab_size) }

## Step 3 - Have a look !
#### Show embedding of a word :

In [7]:
word_embedding['nice']

array([-0.34231952,  0.2524404 , -0.26912802,  0.32358423,  0.27203482,
       -0.1726242 , -0.24930558, -0.27215916,  0.25979856,  0.3433097 ,
        0.28283167,  0.30599555, -0.3370905 , -0.3118096 ,  0.23309754,
        0.24294853], dtype=float32)

#### Few usefull functions to play with

In [8]:
# Return a l2 distance between 2 words
#
def l2w(w1,w2):
    v1=word_embedding[w1]
    v2=word_embedding[w2]
    return np.linalg.norm(v2-v1)

# Show distance between 2 words 
#
def show_l2(w1,w2):
    print(f'\nL2 between [{w1}] and [{w2}] : ',l2w(w1,w2))

# Displays the 15 closest words to a given word
#
def neighbors(w1):
    v1=word_embedding[w1]
    dd={}
    for i in range(4, 1000):
        w2=index_word[i]
        dd[w2]=l2w(w1,w2)
    dd= {k: v for k, v in sorted(dd.items(), key=lambda item: item[1])}
    print(f'\nNeighbors of [{w1}] : ', list(dd.keys())[1:15])
    

#### Examples

In [9]:
show_l2('nice', 'pleasant')
show_l2('nice', 'horrible')

neighbors('horrible')
neighbors('great')



L2 between [nice] and [pleasant] :  0.738236

L2 between [nice] and [horrible] :  4.0432134

Neighbors of [horrible] :  ['badly', 'worse', 'avoid', 'ridiculous', 'fails', 'mess', 'terrible', 'poor', 'save', 'annoying', 'predictable', 'dull', 'poorly', 'boring']

Neighbors of [great] :  ['9', 'amazing', 'perfectly', 'highly', 'wonderful', 'loved', 'fun', 'enjoyed', 'fantastic', 'definitely', 'brilliant', 'enjoyable', 'today', 'superb']
